In [3]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import shapiro, levene
import matplotlib.pyplot as plt

In [4]:
# Создаем DataFrame из данных
data = pd.DataFrame({
    'A': [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
    'B': [1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2],
    'Отклик': [3.33, 2.75, 3.32, 1.44, 4.41, 5.64, 6.96, 8.15, 6.99, 5.60, 5.02, 5.01, 3.24, 4.10, 5.12, 3.16, 3.84, 3.82, 6.51, 6.25, 6.61, 7.17, 6.92, 4.10, 8.21, 7.83, 5.36, 7.86]
})


In [5]:

# Построение линейной модели с взаимодействием
model = ols('Отклик ~ C(A) + C(B) + C(A):C(B)', data=data).fit()

# Многофакторный ANOVA с III типом разложения сумм квадратов
anova_table = sm.stats.anova_lm(model, typ=3)
print("Многофакторный ANOVA (тип III):")
print(anova_table)


Многофакторный ANOVA (тип III):
              sum_sq    df          F    PR(>F)
Intercept  46.512500   1.0  42.180789  0.000002
C(A)       30.241873   2.0  13.712723  0.000136
C(B)       28.033333   1.0  25.422588  0.000048
C(A):C(B)  33.697931   2.0  15.279821  0.000069
Residual   24.259267  22.0        NaN       NaN


In [6]:

# Проверка значимости факторов и взаимодействия
if anova_table['PR(>F)']['C(A)'] < 0.05:
    print("\nФактор A значим. Проводим попарное сравнение для A:")
    tukey_a = pairwise_tukeyhsd(endog=data['Отклик'], groups=data['A'], alpha=0.05)
    print(tukey_a)

if anova_table['PR(>F)']['C(B)'] < 0.05:
    print("\nФактор B значим. Проводим попарное сравнение для B:")
    tukey_b = pairwise_tukeyhsd(endog=data['Отклик'], groups=data['B'], alpha=0.05)
    print(tukey_b)

if anova_table['PR(>F)']['C(A):C(B)'] < 0.05:
    print("\nВзаимодействие A и B значимо. Проводим попарное сравнение для взаимодействия:")
    
    data['A_B'] = data['A'].astype(str) + "_" + data['B'].astype(str)
    tukey_interaction = pairwise_tukeyhsd(endog=data['Отклик'], groups=data['A_B'], alpha=0.05)
    print(tukey_interaction)


Фактор A значим. Проводим попарное сравнение для A:
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     1      2     0.09 0.9921 -1.7785 1.9585  False
     1      3    2.182 0.0198  0.3135 4.0505   True
     2      3    2.092 0.0177  0.3304 3.8536   True
---------------------------------------------------

Фактор B значим. Проводим попарное сравнение для B:
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     1      2    0.609 0.3935 -0.8337 2.0517  False
---------------------------------------------------

Взаимодействие A и B значимо. Проводим попарное сравнение для взаимодействия:
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
   1_1    1_2   3.8667 0.0006  1